<a href="https://colab.research.google.com/github/aharoncooper/immune-cell-classification/blob/main/training_yolov8_on_immune_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the ultralytics package for YOLOv8
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 13.2 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("smfaizanahmed/wbc-object-detection-dataset-yolov8")

print("Path to dataset files:", path)

100%|██████████| 249M/249M [00:01<00:00, 168MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/smfaizanahmed/wbc-object-detection-dataset-yolov8/versions/1


In [ ]:
!kaggle datasets download -d smfaizanahmed/wbc-object-detection-dataset-yolov8
!unzip wbc-object-detection-dataset-yolov8.zip -d /content/dataset


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-36-_jpg.rf.cc6075947b9a1614f5d513976a5143b3.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-37-_jpg.rf.22f77500b0c77fcc14dca20e0e86fadd.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-37-_jpg.rf.4360a989ce4ee6ebc1bc0b239eb1d816.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-37-_jpg.rf.99173b5bbc475dc3ee5ed54a17331d01.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-38-_jpg.rf.6a69e543041c0527cff9405aeab051be.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-38-_jpg.rf.9d4757a64b69cf6282e437a7a6c2c94a.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-39-_jpg.rf.79a338fe7657b1a2e1e613bef5d453ef.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-39-_jpg.rf.97f1dae4a0e37bbc7cadea2348bbae49.txt  
  inflating: /content/dataset/CBC(WBC)/train/labels/Mono-4-_jpg.rf.3ca9a8858ad23136d29d616b89

In [ ]:
import os

# Specify the path to the directory containing the annotation files
label_dir = '/content/dataset/CBC(WBC)/train/labels'  # Adjust path as needed

# Set to store unique class IDs
unique_class_ids = set()

# Loop through each annotation file
for file_name in os.listdir(label_dir):
    if file_name.endswith('.txt'):  # Ensure only .txt files are processed
        with open(os.path.join(label_dir, file_name), 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_id = int(line.split()[0])  # Extract the class ID (first value in the line)
                unique_class_ids.add(class_id)

# Print unique class IDs found in the annotation files
print("Unique class IDs found in annotations:", unique_class_ids)


Unique class IDs found in annotations: {1}


In [ ]:
import os

# Paths to the directories containing training and validation annotation files
train_label_dir = '/content/dataset/CBC(WBC)/train/labels'  # Adjust if necessary
valid_label_dir = '/content/dataset/CBC(WBC)/valid/labels'      # Adjust if necessary

def update_class_ids(label_dir):
    """
    Updates class IDs from 1 to 0 in all annotation files in the specified directory.
    """
    for file_name in os.listdir(label_dir):
        if file_name.endswith('.txt'):  # Ensure only .txt files are processed
            file_path = os.path.join(label_dir, file_name)
            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Replace class ID 1 with 0
            updated_lines = []
            for line in lines:
                parts = line.split()
                if parts[0] == '1':  # Check if class ID is 1
                    parts[0] = '0'  # Replace with 0
                updated_lines.append(' '.join(parts))

            # Write the updated content back to the file
            with open(file_path, 'w') as f:
                f.writelines("\n".join(updated_lines))

    print(f"Updated all annotation files in {label_dir} to use class ID 0.")

# Update both training and validation annotations
update_class_ids(train_label_dir)
update_class_ids(valid_label_dir)


Updated all annotation files in /content/dataset/CBC(WBC)/train/labels to use class ID 0.
Updated all annotation files in /content/dataset/CBC(WBC)/valid/labels to use class ID 0.


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model (e.g., YOLOv8n for a smaller model)
model = YOLO('yolov8n.pt')  # You can replace with 'yolov8s.pt', 'yolov8m.pt', or 'yolov8l.pt'

# Train the model on your dataset
model.train(data='/content/dataset/CBC(WBC)/data.yaml', epochs=50, imgsz=640, batch=16)


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/CBC(WBC)/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning /content/dataset/CBC(WBC)/train/labels... 7655 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7655/7655 [00:05<00:00, 1472.93it/s]


train: New cache created: /content/dataset/CBC(WBC)/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/dataset/CBC(WBC)/valid/labels... 1931 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1931/1931 [00:01<00:00, 1308.83it/s]


val: New cache created: /content/dataset/CBC(WBC)/valid/labels.cache
Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.328      2.863      1.666         29        640:   7%|▋         | 33/479 [07:27<1:40:42, 13.55s/it]


KeyboardInterrupt: 

In [ ]:
model.save('/content/yolov8_best.pt')


In [ ]:
from google.colab import files
files.download('/content/yolov8_best.pt')


In [ ]:
# Display training results
model.plot_results()


In [ ]:
# Run inference on a sample image
results = model.predict(source='/content/3666-Normal peripheral blood smear - 1.JPG', conf=0.1)

# Check the type of `results`
if isinstance(results, list):
    results = results[0]  # Access the first result if `results` is a list

# Display results
results.show()


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8_best.pt')  # Replace with your trained model file

# Load the large image
image_path = '//content/3666-Normal peripheral blood smear - 1.JPG'
large_image = cv2.imread(image_path)
if large_image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Parameters for tiling
tile_size = 640  # Match the size used during training
stride = 320  # Overlap between tiles to avoid missing objects

# Get image dimensions
height, width, _ = large_image.shape

# Create a blank canvas to visualize detections
output_image = large_image.copy()

# List to store detections
detections = []

# Process each tile
for y in range(0, height, stride):
    for x in range(0, width, stride):
        # Extract the tile
        tile = large_image[y:y + tile_size, x:x + tile_size]

        # Skip incomplete tiles at the edges
        if tile.shape[0] != tile_size or tile.shape[1] != tile_size:
            continue

        # Run YOLO detection on the tile
        results = model.predict(source=tile, conf=0.4, verbose=False)

              # Process detections
        for result in results:  # Iterate through the results
            if hasattr(result, 'boxes') and result.boxes is not None:
                boxes = result.boxes.cpu().numpy()  # Convert boxes to numpy array for processing

                # Print to check the structure of boxes
                print(f"Boxes: {boxes}")

                for box in boxes:
                    if len(box) >= 4:  # Ensure there are enough elements for coordinates
                        x_min, y_min, x_max, y_max = box[:4]
                        # The remaining elements should be confidence and class ID if present
                        if len(box) >= 6:
                            confidence = box[4]
                            class_id = int(box[5])

                            # Adjust box coordinates relative to the full image
                            adjusted_box = [
                                int(x_min + x),
                                int(y_min + y),
                                int(x_max + x),
                                int(y_max + y),
                                confidence,
                                class_id
                            ]
                            detections.append(adjusted_box)

                            # Draw the box on the output image
                            cv2.rectangle(
                                output_image,
                                (adjusted_box[0], adjusted_box[1]),
                                (adjusted_box[2], adjusted_box[3]),
                                color=(0, 255, 0),  # Green for boxes
                                thickness=2
                            )
                            label = f"Class {class_id} {confidence:.2f}"
                            cv2.putText(
                                output_image, label,
                                (adjusted_box[0], adjusted_box[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.5, (0, 255, 0), 1
                            )
                        else:
                            print("Box does not have confidence or class ID.")
                    else:
                        print("Box does not have enough data.")

# Save or display the output
output_path = '/content/output_image_with_detections.jpg'
cv2.imwrite(output_path, output_image)
print(f"Detections saved to {output_path}")


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Function to display an image using OpenCV or matplotlib
def show_image(image, title="Image"):
    # Convert BGR to RGB for displaying with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Assuming `output_image` is the image with detections drawn
show_image(output_image, "Detected White Blood Cells")
